In [1]:
!nvidia-smi

Thu Jan 18 06:51:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
!pip install -q transformers[sentencepiece] ohmeow-blurr fastai nbdev

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.1/66.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [4]:
from google.colab import drive
drive.mount('/content/drive', timeout_ms=3000000)

Mounted at /content/drive


In [9]:
%cd /content/drive/MyDrive/keyword_categorization/

/content/drive/.shortcut-targets-by-id/18aTrjN2Ugh0bCahBW3L1xSwAl-MNEXVD/keyword_categorization


In [10]:
import pandas as pd
from fastai.text.all import *

In [11]:
df = pd.read_csv("data/papers_final_data.csv")

In [12]:
for index in range(len(df)):
  df.iloc[index]['final_keywords'] = [ item.replace(" '","").replace("'","").lower() for item in re.split(",", df.iloc[index]['final_keywords'].replace("[","").replace("]","")) ]

In [13]:
available_keywords = {}

keywords_list = df['final_keywords'].to_list()

for item in range(len(keywords_list)):
  for keyword in keywords_list[item]:
    if keyword in available_keywords.keys():
      available_keywords[keyword] += 1
    else:
      available_keywords[keyword] = 1

In [14]:
encoding_path = "encoding"

In [15]:
encode_keywords = { key: idx for idx, (key, value) in enumerate(available_keywords.items())}
with open(f"{encoding_path}/encoded_keywords.json", "w") as fp:
  json.dump(encode_keywords, fp)

In [16]:
categorical_keywords_list = []

for keywords in keywords_list:
  categorical_list = [0]*len(encode_keywords)
  for keyword in keywords:
    keyword_index = encode_keywords[keyword]
    categorical_list[keyword_index] = 1
  categorical_keywords_list.append(categorical_list)

In [17]:
df['encoded_keywords'] = categorical_keywords_list

In [18]:
df.tail()

,abstracts,final_keywords,encoded_keywords
36393,"Wafer level packaging could reduce the cost of MEMS based sensors through simplified processing and inexpensive components. A novel vacuum tight seal referred to as the wedge seal method is proposed in this paper. The seal consists of a silicon wedge forced into a pliable material (typically a metal) that is attached to the component wafer. The wedge-seal addresses some of the requirements of micro-bolometer packaging in that it provides a vacuum tight seal at low temperatures with tolerance to surface finish and topography. The concept was evaluated with regards to manufacturability, mate...","[silicon, sensors]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
36394,"In practical image storage and distribution systems. JPEG is a common image format. In this paper, we propose a blind watermarking method for JPEG-compressed color images in the semi-decompression domain by integrating the JPEG coding process. The watermark is embedded by modifying the quantized DC coefficients based on the spread spectrum technique. There are some modifications on the basic spread spectrum watermarking scheme to meet the sensitivity of the quantized DC coefficients. Furthermore, Reed-Solomon coding is employed to improve the robustness of watermarking. Experimental result...","[visualization, sensitivity, modulation]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
36395,"This paper reports the effects of the following substrates on the optical, structural and morphological properties of ZnO thin films: Si (100), Si (111), c- and r-sapphire, glass, GaAs and ZnO. The thin films were deposited by low pressure Metal Organic Chemical Vapor Deposition using diethylzinc and tertiary butanol as the zinc and oxygen sources, respectively. X-ray diffraction analysis shows that all the ZnO layers are c-axis orientated. Scanning Electron microscopy reveals similar morphology for all the substrates used, with hexagonal columns having cone shape ends being evident. The p...","[substrates, silicon]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
36396,"The stability margin of a two degree-of-freedom self-sensing active magnetic bearing (AMB) is estimated by means of μ-analysis. The specific self-sensing algorithm implemented in this study is the direct current measurement method. Detailed black-box models are developed for the main subsystems in the AMB by means of discrete-time system identification. In order to obtain models for dynamic uncertainty in the various subsystems in the AMB, the identified models are combined to form a closed-loop model for the self-sensing AMB. The response of this closed-loop model is then compared to the ...","[uncertainty, integrated circuit modeling, analytical models, robustness, rotors]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
36397,How learning is conceptualized and negotiated within the engineering undergraduate curriculum is affected by the th

In [19]:
splitter = RandomSplitter(valid_pct = 0.1, seed = 42)
train_ids, valid_ids = splitter(df)

In [20]:
valid_df = df.loc[valid_ids]
valid_df.head()

,abstracts,final_keywords,encoded_keywords
27932,"In TrustZone architecture, the Trusted Application (TA) in the secure world does not certify the identity of Client Applications (CA) in the normal world that request data access, which represents a user data leakage risk. This paper proposes a private user data protection mechanism in TrustZone to avoid such risks. We add corresponding modules to both the secure world and the normal world and authenticate the identity of CA to prevent illegal access to private user data. Then we analyze the system security, and perform validity and performance tests. The results show that this method can ...","[computer architecture, kernel]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
26450,"Synthetic aperture radar is a microwave technique to extracting image information of the target. Electromagnetic waves that are reflected from the target are acquired by the aircraft or satellite receivers and sent to a ground station to be processed by applying computational demanding algorithms. Radar data streams are acquired by an aircraft or satellite and sent to a ground station to be processed in order to extract images from the data since these processing algorithms are computationally demanding. However, novel applications require real-time processing for real-time analysis and de...","[computer architecture, synthetic aperture radar, radar, synthetic aperture radar (sar)]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
19638,"When I heard that next year this Institute is to celebrate fifty years of existence, during which it has grown to more than ten times its original size, and knowing that it would not be possible for me to be with you on that occasion, although I very much hoped at one time that that would be possible, it seemed to me not inappropriate that we might devote our time this evening to a consideration of the past one hundred years, during which the remarkable electrical technology which we know today started from quite simple beginnings. Therefore, Mr President, with your permission, that is the...","[switches, receivers]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
3852,"As recently studied, field-programmable gate arrays (FPGAs) suffer from growing Hardware Trojan (HT) attacks, and many techniques, e.g., register-transfer level (RTL) code-based analyzing, have been presented to detect HTs on FPGAs. However, for most of the FPGA end users, they can only obtain bitstream, rather than the RTL code. Therefore, we present a new FPGA reverse engineering tool-chain. It can precisely transform the FPGA bitstream to an RTL code and therefore assists in HT detection. In detail, we first construct an integrated database involving the FPGA architecture information an...","[computer architecture, switches]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
30734,"The coordinated control of parallel three-phase four-wire converters in autonomous AC microgrids is in

In [21]:
model_name ="roberta.pkl"
# model_name ="bert.pkl"
# model_name ="distilBERT.pkl"

In [22]:
model = load_learner(f"models/{model_name}")

In [23]:
string = "The coordinated control of parallel three-phase four-wire converters in autonomous AC microgrids is investigated in this paper. Firstly, based on droop control, virtual impedance is inserted in positive-, negative- and zero-sequences to enhance system damping and imbalance power sharing. Then, to facilitate virtual impedance design, small signal models of the three-sequence equivalent circuits are established respectively. Corresponding indexes are proposed to evaluate the impact of sequence virtual impedance on current sharing accuracy, voltage quality at the point of common coupling (PCC) and system stability comprehensively. In addition, constraint of dc-link voltage is also considered to avoid over modulation when subjected to unbalanced loads. Furthermore, to address the PCC voltage degradation resulted from the virtual impedance, a voltage imbalance compensation method based on low-bandwidth communication is proposed. Finally, simulation and experimental results are provided to verify the correctness of the theory model, indicating that the proposed method can achieve PCC voltage restoration while guaranteeing the current sharing accuracy with desirable dynamics."

In [ ]:
model.blurr_predict(string)

In [25]:
from tqdm.notebook import tqdm

In [26]:
len(valid_df)

3639

In [27]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df)):
  desc = row['abstracts']
  labels = model.blurr_predict(desc)[0]['class_indices']
  # pred_genres = [0] * len(encode_genre_types)
  # for label in labels:
  #   pred_genres[encode_genre_types[label]] = 1
  preds.append(labels)

preds[0][:20]

  0%|          | 0/3639 [00:00<?, ?it/s]

[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [28]:
from sklearn import metrics

def metric_measures(test_df, preds, model_name):

  targets = [np.asarray(target) for target in test_df['encoded_keywords'].to_list()]

  # print(f"Targets: {targets[0]}, Datatype: {type(targets[0])}")

  outputs = [np.asarray(pred) for pred in preds]

  # print(f"Outputs: {outputs[0]}, Datatype: {type(outputs[0])}")

  accuracy = metrics.accuracy_score(targets, outputs)

  f1_score_micro = metrics.f1_score(targets, outputs, average = 'micro')

  f1_score_macro = metrics.f1_score(targets, outputs, average = 'macro')

  print(f"{model_name}'s evaluation metrics:")

  print(f"Clasification matrix:")
  print(f"{metrics.classification_report(targets, outputs, digits=6)}")

  print(f"\tOverall Accuracy: {accuracy*100}%")

  # Micro F1-score is more accurate for multi-label classification
  print(f"\tF1 Score (Micro) = {f1_score_micro*100}%")

  # Macro F1-score is more accurate for single-label classification
  print(f"\tF1 Score (Macro) = {f1_score_macro*100}%")

  return

<h2><strong>RoBERTa</strong></h2>

In [29]:
metric_measures(valid_df, preds, "RoBERTa")

RoBERTa's evaluation metrics:
Clasification matrix:
              precision    recall  f1-score   support

           0   0.666667  0.454545  0.540541        88
           1   0.764706  0.406250  0.530612        32
           2   0.921053  0.921053  0.921053        38
           3   0.618182  0.225166  0.330097       151
           4   0.800000  0.093023  0.166667        43
           5   1.000000  0.024390  0.047619        41
           6   0.540541  0.238095  0.330579        84
           7   0.648649  0.406780  0.500000        59
           8   0.839286  0.610390  0.706767        77
           9   0.551724  0.134454  0.216216       119
          10   0.740741  0.487805  0.588235        82
          11   0.500000  0.121212  0.195122        33
          12   0.666667  0.025641  0.049383        78
          13   0.409091  0.195652  0.264706        46
          14   0.702479  0.615942  0.656371       138
          15   0.500000  0.046053  0.084337       152
          16   1.000000  0.01

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<h2><strong>BERT</strong></h2>

In [ ]:
metric_measures(valid_df, preds, "BERT")

BERT's evaluation metrics:
Clasification matrix:               precision    recall  f1-score   support

           0   0.658537  0.613636  0.635294        88
           1   0.766667  0.718750  0.741935        32
           2   0.891892  0.868421  0.880000        38
           3   0.609524  0.423841  0.500000       151
           4   0.562500  0.418605  0.480000        43
           5   0.454545  0.121951  0.192308        41
           6   0.472222  0.404762  0.435897        84
           7   0.704918  0.728814  0.716667        59
           8   0.812500  0.675325  0.737589        77
           9   0.615385  0.470588  0.533333       119
          10   0.726027  0.646341  0.683871        82
          11   0.500000  0.424242  0.459016        33
          12   0.583333  0.358974  0.444444        78
          13   0.739130  0.369565  0.492754        46
          14   0.690647  0.695652  0.693141       138
          15   0.660714  0.243421  0.355769       152
          16   0.448276  0.16250

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **DistilBERT**

In [ ]:
metric_measures(valid_df, preds, "DistilBERT")

DistilBERT's evaluation metrics:
Clasification matrix:               precision    recall  f1-score   support

           0   0.716216  0.602273  0.654321        88
           1   0.758621  0.687500  0.721311        32
           2   0.894737  0.894737  0.894737        38
           3   0.628205  0.324503  0.427948       151
           4   0.333333  0.116279  0.172414        43
           5   0.857143  0.146341  0.250000        41
           6   0.584615  0.452381  0.510067        84
           7   0.709091  0.661017  0.684211        59
           8   0.796875  0.662338  0.723404        77
           9   0.545455  0.403361  0.463768       119
          10   0.742424  0.597561  0.662162        82
          11   0.565217  0.393939  0.464286        33
          12   0.568182  0.320513  0.409836        78
          13   0.612903  0.413043  0.493506        46
          14   0.723577  0.644928  0.681992       138
          15   0.744186  0.210526  0.328205       152
          16   0.538462  0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
